![](epn.png "titulo")


                                                  **ESCUELA POLITÉCNICA NACIONAL**     


                                                **FACULTAD DE INGENIERÍA DE SISTEMAS** 

_______________________________________________________________________________________________________________________________

**PERÍODO ACADÉMICO:**2019-B

**ASIGNATURA:** PYTHON

**PROFESOR:** Ing. Adrián Eguez

**GRUPO:** GR1

**FECHA:** 2019-02-02

**Alumno:** Cerón Laura

_______________________________________________________________________________________________________________________________


# Tema: Productos Farmacéuticos

# Introducción
<p style="text-align: justify;">

En Ecuador, la industria farmacéutica atiende a dos sectores; el primero de ellos es el del mercado privado, cuyos medicamentos se venden en las farmacias, clínicas privadas y cadenas farmacéuticas; y el segundo es el de salud pública, el cual abastece a través de las compras públicas de fármacos.

Al ser las farmacias el canal de distribución final para los laboratorios, las estrategias para la distribución y venta al cliente se orientan fuertemente a este tipo de cadenas de negocios, las mismas que han mostrado un crecimiento a partir de la ampliación de sus servicios.

Por ello se realizo webscrapy a la farmacia fybeca y a la farmacia medicity, con el fin de comparar los precios de algunos productos.

</p>

# Descripción de las columnas

Para desarrollo del presente informe se usa dos archivos json producto del web crawling.

A continuación se presenta una tabla el nombre y la descripción de cada uno de los archivos:


| Nombre columna| Descripción                                       |
|:------------- |:----------------------------------------------    |
|titulo	        |Nombre del producto                                |
|imagen	        |Url de la imagen del producto	                    |
|precio	        |Precio del producto	                            |
|categoria	    |categoria del producto (medicina, vitaminas, etc)  | 
|farmacia	    |Nombre de la farmacia                              | 


# Desarrollo.



## Scraping y Web Crawling 

A continuación se hara presentara el codigo utilizado para realizar el web crawling.

In [ ]:
import scrapy
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor
from scrapy.loader import ItemLoader
from arania_medicity.items import ProductoMedicity
from scrapy.loader.processors import TakeFirst
class AraniaMedicity(CrawlSpider):
    name = 'arania_medicity'  
    start_urls = ['https://www.farmaciasmedicity.com/shop/category/medicinas-1328']
    allowed_domains = ['farmaciasmedicity.com']
    regla_uno = (Rule( LinkExtractor(), callback='parse_page'),)
    url_segmento_permitido = ('category')
    regla_dos = ( Rule(LinkExtractor(allow_domains=allowed_domains,allow = url_segmento_permitido), callback='parse_page'),)   
    rules = regla_dos
    def parse_page(self, response):        
        productos = response.xpath('//div[@id="grid_list"]')
        categoria_dato=response.request.url.split('/')                
        categoria = categoria_dato[5]       
        tiene_categoria = False      
        if "cuidado-personal-y-belleza" in categoria:
            tiene_categoria = True        
        if(tiene_categoria):
            for producto in productos:
                producto_loader =ItemLoader(item = ProductoMedicity(),selector = producto)
                producto_loader.default_output_processor = TakeFirst()                               
                producto_loader.add_css('titulo', 'a::text')
                producto_loader.add_css('imagen', 'img::attr(data-zoom-image)')
                producto_loader.add_css('precio', 'span.oe_currency_value::text')                 
                producto_loader.add_css('categoria','span.oe_currency_value::text')
                producto_loader.add_css('farmacia','span.oe_currency_value::text')                    
                yield producto_loader.load_item()         

In [ ]:
import scrapy
from scrapy.loader.processors import TakeFirst, MapCompose

def transformar_url_imagen(texto):        
    url_medicity = 'https://www.farmaciasmedicity.com'   
    return url_medicity+texto
def transformar_precio(texto):     
    return texto
def transformar_titulo(texto):   
    if not(len(texto) == 21 or len(texto) == 17):
        return texto
def transformar_categoria(texto):   
    return "Belleza"      
def transformar_farmacia(texto):     
    return "Medicity"      
class ProductoMedicity(scrapy.Item):        
    titulo = scrapy.Field(input_processor = MapCompose(transformar_titulo), output_processor =  TakeFirst())
    imagen = scrapy.Field(input_processor = MapCompose(transformar_url_imagen), output_processor = TakeFirst())
    precio = scrapy.Field(input_processor = MapCompose(transformar_precio), output_processor = TakeFirst())
    categoria = scrapy.Field(input_processor = MapCompose(transformar_categoria), output_processor = TakeFirst())
    farmacia = scrapy.Field(input_processor = MapCompose(transformar_farmacia), output_processor = TakeFirst())
class AraniaMedicityItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    pass

## Análisis de datos.
Una vez obtenido los datos se prodece con el análisis.
#### Librerias utilizadas.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pylab as pl

#### Lectura de archivos.